# BackPropagation算法的矩阵形式推导

## 符号说明
|符号|说明|
|:|:|
|$d^{(l)}$|第$l$层的节点个数|
| $W^{(l)}$|$d^{(l)}\times d^{(l-1)}$的权值矩阵，$W^{(l)}_{ij}$表示第$l-1$层节点$j$到第$l$层节点$i$的突触权值|
| $x^{(l)}$|第$l$层每个节点输出组成的向量，是一个$d^{(l)}\times 1$的列向量|
|$s^{(l)}$|第$l$层每个节点输入组成的向量，同上|
|$b^{(l)}$|第$l$层每个节点的偏置，同上|
| $\sigma(x)$|_sigmoid_函数，$\sigma(x)=1/(1+exp(-x))$，它的导数是$\sigma^\prime(x)=\sigma(x)(1-\sigma(x))$|
| $n$|样本数|
| $J(W,b;x_i,y_i)$|当输入为$(x_i,y_i)$时的损失函数，以下简记为$J$|

# 前馈过程和目标函数
神经网络的前馈公式：
$$ \left\{ \begin{aligned} & s^{(l)} = W^{(l)} \cdot x^{(l-1)}  + b^{(l)}\\ & x^{(l)} = \sigma(s^{(l)})  \end{aligned} \right. $$

神经网络的目标是寻找使得如下目标函数最大的参数：
$$ J(\theta) =  \frac{1}{n} \sum_{i=1}^n \left\|y_i - \hat{y}_i\right\|^2$$
其中，$\theta$是参数集合，$\hat{y}_i=\sigma(W^{(L)}x_i^{(L-1)}+ b^{(L)})$是神经网络的输出

## BP算法推导
BP算法的关键是计算$\delta^{(l)}=\frac{\partial J}{\partial s^{(l)}}$，其含义是最终误差对于第$l$层神经元的敏感程度。
依据链式法则，有
$$ \begin{aligned}\delta^{(l)}=\frac{\partial J}{\partial s^{(l)}} = &( \frac{\partial J}{\partial s^{(l+1)}} \cdot \frac{\partial s^{(l+1)}}{\partial x^{(l)}} \cdot\frac{\partial x^{(l)}}{\partial s^{(l)}} ) ^T\\ = &\{ (\delta^{(l+1)})^T \cdot W^{(l+1)} \cdot diag(\sigma^\prime(s^{(l)}))\}^T\\=& diag(\sigma^\prime(s^{(l)}))\cdot (W^{(l+1)})^T\delta^{(l+1)}\\=& \sigma^\prime(s^{(l)})\odot ((W^{(l+1)})^T\delta^{(l+1)})\end{aligned}$$
其中$\odot$是向量点积。于是我们有了误差敏感度的递推公式：
$$ \delta^{(l)} = \sigma^\prime(s^{(l)})\odot ((W^{(l+1)})^T\delta^{(l+1)})$$
于是，梯度为
$$ \nabla{W}^{(l)}=\frac{\partial J}{\partial W^{(l)}}=\delta^{(l)}\otimes x^{(l-1)}$$
$$ W^{(l)}\longleftarrow W^{(l)} - \alpha  \nabla{W}^{(l)}$$